In [57]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2016 movies from Wikipedia

In [58]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2016"

In [59]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [60]:
tables = soup.find_all('table',class_='wikitable sortable')

In [61]:
len(tables)

4

In [62]:
type(tables[0])

bs4.element.Tag

In [63]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [64]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [65]:
df

,Opening,Opening.1,Title,Production Company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Production company,Ref.
0,J A N U A R Y,8.0,The Forest,Focus Features,"Jason Zada (director); Natalie Dormer, Taylor ...",[2],NaN,NaN
1,J A N U A R Y,8.0,Anesthesia,IFC Films,"Tim Blake Nelson (director); Sam Waterston, Ti...",[3],NaN,NaN
2,J A N U A R Y,8.0,Lamb,The Orchard,"Ross Partridge (director); Ross Partridge, Oon...",[4],NaN,NaN
3,J A N U A R Y,15.0,Ride Along 2,Universal Pictures,"Tim Story (director); Ice Cube, Kevin Hart",[5],NaN,NaN
4,J A N U A R Y,15.0,13 Hours: The Secret Soldiers of Benghazi,Paramount Pictures,"Michael Bay (director); James Badge Dale, John...",[6],NaN,NaN
...,...,...,...,...,...,...,...,...
178,D E C E M B E R,23.0,Silence,NaN,Martin Scorsese (director); Jay Cocks (screenp...,NaN,Paramount Pictures,[178]
179,D E C E M B E R,23.0,Why Him?,NaN,"John Hamburg (director, screenwriter); Ian Hel...",NaN,20th Century Fox / 21 Laps Entertainment / Red...,[179]
180,D E C E M B E R,25.0,Hidden Figures,NaN,Theodore Melfi (director); Allison Schroeder (...,NaN,20th Century Fox / Chernin Entertainment,[180]
181,D E C E M B E R,25.0,Live by Night,NaN,Ben Affleck (director/screenplay); Ben Affleck...,NaN,Warner Bros. Pictures / RatPac Entertainment,[181]


In [66]:
df_2016 = df[['Title','Cast and crew']]

In [67]:
df_2016

,Title,Cast and crew
0,The Forest,"Jason Zada (director); Natalie Dormer, Taylor ..."
1,Anesthesia,"Tim Blake Nelson (director); Sam Waterston, Ti..."
2,Lamb,"Ross Partridge (director); Ross Partridge, Oon..."
3,Ride Along 2,"Tim Story (director); Ice Cube, Kevin Hart"
4,13 Hours: The Secret Soldiers of Benghazi,"Michael Bay (director); James Badge Dale, John..."
...,...,...
178,Silence,Martin Scorsese (director); Jay Cocks (screenp...
179,Why Him?,"John Hamburg (director, screenwriter); Ian Hel..."
180,Hidden Figures,Theodore Melfi (director); Allison Schroeder (...
181,Live by Night,Ben Affleck (director/screenplay); Ben Affleck...


In [68]:
!pip install tmdbv3api

Defaulting to user installation because normal site-packages is not writeable


In [69]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '71b41c0bf036bd0a8917fbc020e0a79d'

In [70]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [71]:
df_2016['genres'] = df_2016['Title'].map(lambda x: get_genre(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\2205787401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['genres'] = df_2016['Title'].map(lambda x: get_genre(str(x)))


In [72]:
df_2016

,Title,Cast and crew,genres
0,The Forest,"Jason Zada (director); Natalie Dormer, Taylor ...",Horror Thriller Mystery
1,Anesthesia,"Tim Blake Nelson (director); Sam Waterston, Ti...",Drama Thriller
2,Lamb,"Ross Partridge (director); Ross Partridge, Oon...",Horror Drama Thriller
3,Ride Along 2,"Tim Story (director); Ice Cube, Kevin Hart",Action Comedy
4,13 Hours: The Secret Soldiers of Benghazi,"Michael Bay (director); James Badge Dale, John...",War Action History Drama
...,...,...,...
178,Silence,Martin Scorsese (director); Jay Cocks (screenp...,Horror
179,Why Him?,"John Hamburg (director, screenwriter); Ian Hel...",Comedy
180,Hidden Figures,Theodore Melfi (director); Allison Schroeder (...,Drama History
181,Live by Night,Ben Affleck (director/screenplay); Ben Affleck...,Crime Drama Thriller


In [73]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [74]:
df_2016['director_name'] = df_2016['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\2336725051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['director_name'] = df_2016['Cast and crew'].map(lambda x: get_director(str(x)))


In [75]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [76]:
df_2016['actor_1_name'] = df_2016['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\4171489004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['actor_1_name'] = df_2016['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [77]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [78]:
df_2016['actor_2_name'] = df_2016['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\3164072884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['actor_2_name'] = df_2016['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [79]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [80]:
df_2016['actor_3_name'] = df_2016['Cast and crew'].map(lambda x: get_actor3(str(x)))

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\2970458413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['actor_3_name'] = df_2016['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [81]:
df_2016

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Forest,"Jason Zada (director); Natalie Dormer, Taylor ...",Horror Thriller Mystery,Jason Zada,Jason Zada (director); Natalie Dormer,Taylor Kitsch,NaN
1,Anesthesia,"Tim Blake Nelson (director); Sam Waterston, Ti...",Drama Thriller,Tim Blake Nelson,Tim Blake Nelson (director); Sam Waterston,Tim Blake Nelson,Kristen Stewart
2,Lamb,"Ross Partridge (director); Ross Partridge, Oon...",Horror Drama Thriller,Ross Partridge,Ross Partridge (director); Ross Partridge,Oona Laurence,Jess Weixler
3,Ride Along 2,"Tim Story (director); Ice Cube, Kevin Hart",Action Comedy,Tim Story,Tim Story (director); Ice Cube,Kevin Hart,NaN
4,13 Hours: The Secret Soldiers of Benghazi,"Michael Bay (director); James Badge Dale, John...",War Action History Drama,Michael Bay,Michael Bay (director); James Badge Dale,John Krasinski,Max Martini
...,...,...,...,...,...,...,...
178,Silence,Martin Scorsese (director); Jay Cocks (screenp...,Horror,Martin Scorsese,Liam Neeson,Andrew Garfield,Adam Driver
179,Why Him?,"John Hamburg (director, screenwriter); Ian Hel...",Comedy,"John Hamburg (director, screenwriter); Ian Hel...",James Franco,Bryan Cranston,Zoey Deutch
180,Hidden Figures,Theodore Melfi (director); Allison Schroeder (...,Drama History,Theodore Melfi,Taraji P. Henson,Octavia Spencer,Kevin Costner
181,Live by Night,Ben Affleck (director/screenplay); Ben Affleck...,Crime Drama Thriller,Ben Affleck,Ben Affleck,Sienna Miller,Chris Messina


In [82]:
df_2016 = df_2016.rename(columns={'Title':'movie_title'})

In [83]:
new_df20 = df_2016.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [84]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Jason Zada,Jason Zada (director); Natalie Dormer,Taylor Kitsch,NaN,Horror Thriller Mystery,The Forest
1,Tim Blake Nelson,Tim Blake Nelson (director); Sam Waterston,Tim Blake Nelson,Kristen Stewart,Drama Thriller,Anesthesia
2,Ross Partridge,Ross Partridge (director); Ross Partridge,Oona Laurence,Jess Weixler,Horror Drama Thriller,Lamb
3,Tim Story,Tim Story (director); Ice Cube,Kevin Hart,NaN,Action Comedy,Ride Along 2
4,Michael Bay,Michael Bay (director); James Badge Dale,John Krasinski,Max Martini,War Action History Drama,13 Hours: The Secret Soldiers of Benghazi
...,...,...,...,...,...,...
178,Martin Scorsese,Liam Neeson,Andrew Garfield,Adam Driver,Horror,Silence
179,"John Hamburg (director, screenwriter); Ian Hel...",James Franco,Bryan Cranston,Zoey Deutch,Comedy,Why Him?
180,Theodore Melfi,Taraji P. Henson,Octavia Spencer,Kevin Costner,Drama History,Hidden Figures
181,Ben Affleck,Ben Affleck,Sienna Miller,Chris Messina,Crime Drama Thriller,Live by Night


In [85]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [86]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      7
actor_3_name     25
genres            2
movie_title       2
comb             25
dtype: int64

In [87]:
new_df20 = new_df20.dropna(how='any')

In [88]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [89]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\2267385682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [90]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
1,Tim Blake Nelson,Tim Blake Nelson (director); Sam Waterston,Tim Blake Nelson,Kristen Stewart,Drama Thriller,anesthesia,Tim Blake Nelson (director); Sam Waterston Tim...
2,Ross Partridge,Ross Partridge (director); Ross Partridge,Oona Laurence,Jess Weixler,Horror Drama Thriller,lamb,Ross Partridge (director); Ross Partridge Oona...
4,Michael Bay,Michael Bay (director); James Badge Dale,John Krasinski,Max Martini,War Action History Drama,13 hours: the secret soldiers of benghazi,Michael Bay (director); James Badge Dale John ...
5,Trevor Wall,Trevor Wall (director); Rob Schneider,Heather Graham,Ken Jeong,Adventure Animation Comedy Family,norm of the north,Trevor Wall (director); Rob Schneider Heather ...
6,Andrew Renzi,Andrew Renzi (director); Richard Gere,Dakota Fanning,Theo James,Drama,the benefactor,Andrew Renzi (director); Richard Gere Dakota F...
...,...,...,...,...,...,...,...
178,Martin Scorsese,Liam Neeson,Andrew Garfield,Adam Driver,Horror,silence,Liam Neeson Andrew Garfield Adam Driver Martin...
179,"John Hamburg (director, screenwriter); Ian Hel...",James Franco,Bryan Cranston,Zoey Deutch,Comedy,why him?,James Franco Bryan Cranston Zoey Deutch John H...
180,Theodore Melfi,Taraji P. Henson,Octavia Spencer,Kevin Costner,Drama History,hidden figures,Taraji P. Henson Octavia Spencer Kevin Costner...
181,Ben Affleck,Ben Affleck,Sienna Miller,Chris Messina,Crime Drama Thriller,live by night,Ben Affleck Sienna Miller Chris Messina Ben Af...


In [91]:
old_df = pd.read_csv('new_data.csv')

In [92]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5359,Jim Strouse,Jessica Williams,Chris O'Dowd,Keith Stanfield,Romance Comedy,the incredible jessica james,Jessica Williams Chris O'Dowd Keith Stanfield ...
5360,Farhad Mann,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Romance,can't buy my love,Adelaide Kane Benjamin Hollingsworth Jean Loui...
5361,Hannaleena Hauru,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Romance Comedy,thick lashes of lauri mäntyvaara,Inka Haapamäki Rosa Honkonen Tiitus Rantala Ha...
5362,Jonathan A. Rosenbaum,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Crime Comedy Action Family,cop and a half: new recruit,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [93]:
final_df = old_df.append(new_df20,ignore_index=True)

C:\Users\Dipanjan Dhar\AppData\Local\Temp\ipykernel_6724\3205022339.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df20,ignore_index=True)


In [94]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5517,Martin Scorsese,Liam Neeson,Andrew Garfield,Adam Driver,Horror,silence,Liam Neeson Andrew Garfield Adam Driver Martin...
5518,"John Hamburg (director, screenwriter); Ian Hel...",James Franco,Bryan Cranston,Zoey Deutch,Comedy,why him?,James Franco Bryan Cranston Zoey Deutch John H...
5519,Theodore Melfi,Taraji P. Henson,Octavia Spencer,Kevin Costner,Drama History,hidden figures,Taraji P. Henson Octavia Spencer Kevin Costner...
5520,Ben Affleck,Ben Affleck,Sienna Miller,Chris Messina,Crime Drama Thriller,live by night,Ben Affleck Sienna Miller Chris Messina Ben Af...


In [95]:
final_df.to_csv('new2_data.csv',index=False)